# Gas Rate Structure
https://www.pge.com/assets/rates/tariffs/grf1024.pdf

In [1]:
# PG&E October 2024 Gas Rate Structure

# Baseline Allowance for Residential Gas Rates (in therms/day)
baseline_allowances = {
    "PGE": {
        "G-1": {
            "territories": {
                "P": {
                    "summer": 0.39,  # therms/day
                    "winter_offpeak": 1.88,
                    "winter_onpeak": 2.19,
                },
                "Q": {
                    "summer": 0.56,
                    "winter_offpeak": 1.48,
                    "winter_onpeak": 2.00,
                },
                "R": {
                    "summer": 0.36,
                    "winter_offpeak": 1.24,
                    "winter_onpeak": 1.81,
                },
                "S": {
                    "summer": 0.39,
                    "winter_offpeak": 1.38,
                    "winter_onpeak": 1.94,
                },
                "T": {
                    "summer": 0.56,
                    "winter_offpeak": 1.31,
                    "winter_onpeak": 1.68,
                },
                "V": {
                    "summer": 0.59,
                    "winter_offpeak": 1.51,
                    "winter_onpeak": 1.71,
                },
                "W": {
                    "summer": 0.39,
                    "winter_offpeak": 1.14,
                    "winter_onpeak": 1.68,
                },
                "X": {
                    "summer": 0.49,
                    "winter_offpeak": 1.48,
                    "winter_onpeak": 2.00,
                },
                "Y": {
                    "summer": 0.72,
                    "winter_offpeak": 2.22,
                    "winter_onpeak": 2.58,
                }
            }
        }
    }
}

# Rate plans for gas (per therm)
rate_plans = {
    "PGE": {
        "G-1": {
            "baseline": {
                "procurement_charge": 0.35402,  # per therm
                "transportation_charge": 1.94995,  # per therm
                "total_charge": 2.30397,  # per therm
            },
            "excess": {
                "procurement_charge": 0.35402,  # per therm
                "transportation_charge": 2.44371,  # per therm
                "total_charge": 2.79773,  # per therm
            }
        },
        "G-PPPS": {  # Public Purpose Program Surcharge
            "residential": 0.11051,  # per therm
        },
        "G-NT": {
            "procurement_charge": 0.27473,
            "transportation_charge": 1.30455,  # based on usage tiers
            "total_charge": 1.57928  # per therm (example tier)
        }
    }
}

Total cost for 50 therms: $115.20


In [19]:
import pandas as pd
from datetime import datetime, timedelta

csv_file_path = "alameda/load/step4_alameda_avg_natural_gas_loads_therms.csv"
load_profile_df = pd.read_csv(csv_file_path)

# Assuming the column 'total_household_load.therms' contains the therms data
load_profile = load_profile_df['total_household_load.therms'].values

def get_season(hour_index):
    """
    Determines the season (summer or winter) for a given hour index.
    
    Args:
        hour_index (int): Hour index (0 to 8759)
    
    Returns:
        str: 'summer' or 'winter'
    """
    start_date = datetime(year=2018, month=1, day=1)  # Same year as NREL inputs
    current_datetime = start_date + timedelta(hours=hour_index)
    month = current_datetime.month

    if 6 <= month <= 9:
        return 'summer'
    else:
        return 'winter'

def calculate_annual_therms(load_profile):
    """
    Calculate the total therms per year and divide them into summer and winter seasons.
    
    Args:
        load_profile (list or np.array): List of 8760 hourly load values in therms.
    
    Returns:
        dict: Total therms for the year, divided into 'summer' and 'winter'.
    """
    total_therms = 0.0
    summer_therms = 0.0
    winter_therms = 0.0
    
    for hour_index in range(8760):
        therms = load_profile[hour_index]
        total_therms += therms
        
        # Add to summer or winter therms
        season = get_season(hour_index)
        if season == 'summer':
            summer_therms += therms
        else:
            winter_therms += therms

    return {
        'total_therms': total_therms,
        'summer_therms': summer_therms,
        'winter_therms': winter_therms
    }

therms_summary = calculate_annual_therms(load_profile)

print(f"Total therms per year: {therms_summary['total_therms']:.2f} therms")
print(f"Total summer therms: {therms_summary['summer_therms']:.2f} therms")
print(f"Total winter therms: {therms_summary['winter_therms']:.2f} therms")

Total therms per year: 110.88 therms
Total summer therms: 31.23 therms
Total winter therms: 79.65 therms


In [20]:
import pandas as pd
from datetime import datetime, timedelta

def get_season(hour_index):
    """
    Determines the season (summer, winter_onpeak, or winter_offpeak) for a given hour index.
    
    Args:
        hour_index (int): Hour index (0 to 8759)
    
    Returns:
        str: 'summer', 'winter_onpeak', or 'winter_offpeak'
    """
    start_date = datetime(year=2018, month=1, day=1)  # Same year as NREL inputs
    current_datetime = start_date + timedelta(hours=hour_index)
    month = current_datetime.month
    hour = current_datetime.hour

    # Summer: June 1 to September 30
    if 6 <= month <= 9:
        return 'summer'
    
    # Winter: October 1 to May 31
    # Define on-peak hours: 6 AM - 10 AM, 5 PM - 9 PM (example on-peak hours)
    if (6 <= hour <= 10) or (17 <= hour <= 21):
        return 'winter_onpeak'
    else:
        return 'winter_offpeak'

def calculate_annual_costs_gas(load_profile, rate_plans, baseline_allowances, territory="T"):
    """
    Calculate the annual gas cost based on the load profile and rate plans.
    
    Args:
        load_profile (list or np.array): List of 8760 hourly load values in therms.
        rate_plans (dict): Dictionary containing gas rate plan details.
        baseline_allowances (dict): Dictionary containing baseline allowances.
        territory (str): The territory for which the rate applies (default is "T").
    
    Returns:
        float: Total annual gas cost.
    """
    total_cost = 0.0
    daily_usage = 0.0

    for hour_index in range(8760):
        season = get_season(hour_index)
        daily_usage += load_profile[hour_index]  # Accumulate daily usage
        
        # Every 24 hours, compare with baseline and reset daily usage
        if (hour_index + 1) % 24 == 0:
            baseline = baseline_allowances["PGE"]["G-1"]["territories"][territory][season]
            
            # Determine if usage is within baseline or exceeds it
            if daily_usage <= baseline:
                rate = rate_plans["PGE"]["G-1"]["baseline"]["total_charge"]
            else:
                rate = rate_plans["PGE"]["G-1"]["excess"]["total_charge"]
            
            # Calculate cost for the day
            daily_cost = daily_usage * rate
            total_cost += daily_cost
            
            # Reset daily usage
            daily_usage = 0.0

    return total_cost

# Path to the 8760 therms load profile
csv_file_path = "alameda/load/step4_alameda_avg_natural_gas_loads_therms.csv"

# Load the therms profile from the CSV
load_profile_df = pd.read_csv(csv_file_path)

# Assuming the total household load in therms is in the column 'total_household_load.therms'
load_profile = load_profile_df['total_household_load.therms'].values 

# Example usage: Calculate annual cost for the therms profile in territory "T"
annual_cost = calculate_annual_costs_gas(load_profile, rate_plans, baseline_allowances, territory="T")
print(f"Total annual gas cost: ${annual_cost:.2f}")

Total annual gas cost: $255.46
